### Imports

In [ ]:
# Needed to read/export in files
import json
import xml.etree.ElementTree as ET
import pandas as pd

# Used for calculations
from tqdm.auto import tqdm
from math import sin, cos, sqrt, atan2, radians
import plotly.figure_factory as ff
import plotly.express as px
from queue import PriorityQueue as PQueue
import numpy as np

# Sanity check plots
import matplotlib.pyplot as plt
from matplotlib import colormaps as cmaps

In [ ]:
with open('../../token.tk') as f:
    px.set_mapbox_access_token(f.readline())
    
px.set_mapbox_access_token('pk.eyJ1IjoibWliYXNlciIsImEiOiJjamphdWZxeTgzMTBuM3BvaGdvdGhidDlzIn0.W6MiurHvSwBs0LvTfEtdrQ')

### Utility methods

In [ ]:
def parse_time(time):
    return int(time[0:2])*3600 + int(time[3:5])*60 + int(time[6:8])

def calc_time(stop1, stop2):
    # assume 1 m/s since we disregard roads so slower walking speed is feasable
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(stop1['lat'])
    lon1 = radians(stop1['lon'])
    lat2 = radians(stop2['lat'])
    lon2 = radians(stop2['lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return int(R * c * 1000)

In [ ]:
# name of folder
FOLDER = 'hsl'

In [ ]:
# The following files are expected to be present and need to be extracted from time tables or similar sources first.

# id : {name, lat, lon}
with open(FOLDER + '/stops.json', 'r') as f:
    stops = json.load(f)

# trip_id : [[arr_time1, dep_time1, stop_id1], ...]
with open(FOLDER + '/trips.json', 'r') as f:
    trips = json.load(f)

# stop_id : [[time1, trip_id1, stop_pos1], ...]
with open(FOLDER + '/stop_data.json', 'r') as f:
    stop_data = json.load(f)
    
# xml of OSM data used for area calculation
tree = ET.parse(FOLDER + '/interpreter')

### Calculation of Hexbin and extraction of coordinates

In [ ]:
root = tree.getroot()

osm_nodes = [] 

for child in root:
    if child.tag != 'node':
        continue
        
    osm_nodes.append({'id': child.attrib['id'],
                      'lat': float(child.attrib['lat']),
                      'lon': float(child.attrib['lon'])})

In [ ]:
# Create a DataFrame
osm_df = pd.DataFrame(osm_nodes)
osm_df = osm_df.set_index('id')
osm_df

In [ ]:
osm_df = osm_df[osm_df['lat'] > 24]
osm_df = osm_df[osm_df['lon'] > 60]

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=osm_df, lat="lat", lon="lon",
    nx_hexagon=120, opacity=0.5, labels={"color": "Point Count"},
    min_count=3,
)

# See if everything looks correct
fig.show()

'Number of Hexagons:', len(fig.data[0].geojson['features'])

In [ ]:
hexagons = fig.data[0].geojson['features']
for ind, feat in enumerate(hexagons):
    
    cords = feat['geometry']['coordinates'][0][:6]
    
    # flip for leaflet
    feat['geometry']['coordinates'][0] = [[i[1], i[0]] for i in cords]
    
    feat['id'] = 'hex_' + str(ind)
    feat['center'] = {'lon': cords[0][0]/2 + cords[3][0]/2, 'lat':cords[0][1]/2 + cords[3][1]/2}
    
with open(FOLDER + '/hexagons.json', 'w') as f:
    json.dump(hexagons, f)

In [ ]:
# clean stops that dont have data
print('stops before:', len(stops))
to_delete = []
for s in stops:
    if s not in stop_data:
        to_delete.append(s)
    else:
        stops[s]['type'] = 'stop'
        

for s in to_delete:
    del stops[s]
print('stops after:', len(stops))

# save number of stops and hexagons
NUM_STOPS = len(stops)

In [ ]:
# append hexagons to stops:
for hexagon in hexagons:
    stops[hexagon['id']] = {'name': hexagon['id'],
                            'type': 'hexagon',
                            'lat': hexagon['center']['lat'],
                            'lon': hexagon['center']['lon']
                           }
    
'Total points:', len(stops)

In [ ]:
MAX_WALK_TIME = 900 # longest time to walk in seconds

distances = {}
seen = set()

for s in stops:
    distances[s] = []

for stop1 in tqdm(stops):
    for stop2 in stops:
        if stop2 in seen:
            continue
        time = calc_time(stops[stop1], stops[stop2])
        if time < MAX_WALK_TIME:
            distances[stop1].append((time, stop2))
            distances[stop2].append((time, stop1))
    
    distances[stop1] = sorted(distances[stop1], key=lambda x: x[0])
    
    seen.add(stop1)
    
    
with open(FOLDER + '/distances.json', 'w') as f:
    json.dump(distances, f)

In [ ]:
# search for a stop
term = 'Kannel'
for s in stops:
    if term in stops[s]['name']:
        print(s, stops[s])

In [ ]:
origin = '1331140' # Aalto metro station

pbar = tqdm(total=len(stops))

best_q_position = {}

for s in stops:
    best_q_position[s] = 1_000_000_000

dist_from_origin = {}
seen = set()
starting_time = parse_time('08:00:00')

q = PQueue()
q.put((starting_time, origin))

while not q.empty():
    time, cur = q.get()
    
    cur = str(cur)
    
    if cur not in seen and cur in stops:
       
        seen.add(cur)
        pbar.update(1)
        #pbar.set_description("Queue size %u" % q.qsize())
        dist_from_origin[cur] = time - starting_time
    
        # add all unseen stops in walking distance
        for walk_time, n_id in distances[cur][1:]:
            temp = str(n_id)
            if temp not in seen and (time + walk_time) < best_q_position[temp]:
                q.put((time + walk_time, temp))
                best_q_position[temp] = time + walk_time

        # for a stop: add all trips leaving station
        if stops[cur]['type'] == 'stop':
            for dep_time, trip_id, pos in stop_data[cur]:
                if dep_time < time:
                    continue

                # get all further stops on the trip and add them to the queue
                for arr_time, dep_time, n_id in trips[trip_id][pos-1:]:
                    temp = str(n_id)
                    if temp in stops:
                        if temp not in seen and arr_time < best_q_position[temp]:
                            q.put((arr_time, temp))
                            best_q_position[temp] = arr_time
            
pbar.close()

In [ ]:
initial = {}

for stop in dist_from_origin:
    if stops[stop]['type'] == 'hexagon':
        initial[stop] = dist_from_origin[stop]
        
    
        
with open(FOLDER + '/initial.json', 'w') as f:
    json.dump(initial, f)

In [ ]:
# Check if times look correct
x = []
y = []
c = []

plt.figure(figsize=(12,12))

for s in dist_from_origin:
    if (dist_from_origin[s] < 7200) and stops[s]['type'] == 'hexagon':
        c.append(dist_from_origin[s])
        x.append(stops[s]['lat'])
        y.append(stops[s]['lon'])
    
plt.scatter(y, x, c=c)

In [ ]:
# colormap export

x = np.linspace(0, 1, 30)

colors = cmaps['turbo'](x)[:, :3]
colors = np.floor(colors*255)
colors = list([[int(j) for j in i] for i in colors])

time_map = {}
c = 0
for col in colors:
    time_map[c] = col
    c += 300
    
with open(FOLDER + '/time_map' + '.json', 'w') as f:
    json.dump(time_map, f)
    
time_map